**LetsGrowMore(LGMVIP)- "DATA SCIENCE INTERN "**

**LGMVIP April-22**

**AUTHOR - PARAG PATIL**



**ADVANCED LEVEL TASK**

**TASK 8 : Next Word Prediction**




In [1]:
import numpy as np
from nltk.tokenize import RegexpTokenizer 
from keras.models import Sequential,load_model
from keras.layers import LSTM 
from keras.layers import Dense ,Activation
from tensorflow.keras.optimizers import RMSprop 
import matplotlib.pyplot as plt 
import pickle 
import heapq

In [3]:
text=open(r'1661-0.txt','r',encoding='utf-8').read().lower()
print("corpus length :",len(text))

corpus length : 581888


In [4]:
tokenizer=RegexpTokenizer(r'\w+')
words =tokenizer.tokenize(text)

In [5]:
unique_words=np.unique(words)
unique_words_index=dict((c,i) for i ,c in enumerate(unique_words))

In [6]:
WORD_LENGTH=5
prev_words=[]
next_words=[]
for i in range(len(words)-WORD_LENGTH):
  prev_words.append(words[i:i+WORD_LENGTH])
  next_words.append(words[i+WORD_LENGTH])
print(prev_words[0])
print(next_words[0])


['project', 'gutenberg', 's', 'the', 'adventures']
of


In [7]:
X=np.zeros((len(prev_words),WORD_LENGTH,len(unique_words)),dtype=bool)
Y=np.zeros((len(next_words),len(unique_words)),dtype=bool)
for i ,each_words in enumerate(prev_words):
  for j,each_words in enumerate(each_words):
    X[i,j,unique_words_index[each_words]]=1
  Y[i,unique_words_index[next_words[i]]]=1 

In [8]:
print((X[0][0]))

[False False False ... False False False]


In [9]:
model=Sequential()
model.add(LSTM(128,input_shape=(WORD_LENGTH,len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

In [10]:
optimizer=RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=["accuracy"])
history=model.fit(X,Y,validation_split=0.05,batch_size=128,epochs=2,shuffle=True).history

Epoch 1/2


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


811/811 [==============================] - 259s 316ms/step - loss: 6.0200 - accuracy: 0.1060 - val_loss: 7.0609 - val_accuracy: 0.1003
Epoch 2/2
811/811 [==============================] - 254s 313ms/step - loss: 5.7809 - accuracy: 0.1454 - val_loss: 7.8702 - val_accuracy: 0.1020


In [11]:
model.save('keras_next_word_model.h5')
pickle.dump(history,open('history.p','wb'))
model=load_model('keras_next_word_model.h5')
history=pickle.load(open('history.p','rb'))

In [12]:
def prepare_input(text):
  x=np.zeros((1,WORD_LENGTH,len(unique_words)))
  for t, word in enumerate(text.split()):
    print(word)
    x[0,t,unique_words_index[word]]=1
  return x
prepare_input("How are you ".lower())

how
are
you


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [13]:
def sample(preds,top_n=3):
  preds=np.asarray(preds).astype("float64")
  preds=np.log(preds)
  exp_preds=np.exp(preds)
  preds=exp_preds / np.sum(exp_preds)

  return heapq.nlargest(top_n, range(len(preds)),preds.take)
  

In [14]:
def predict_completions(text,n=3):
  if text=="":
    return('0')
  x=prepare_input(text)
  preds=model.predict(X,verbose=0)[0]
  next_indices=sample(preds,n)
  return [unique_words[inx] for idx in next_indices]

In [ ]:
q="Do your work by your own instead of depending on someone "
print("correct sentence : ",q)
seq=" ".join(tokenizer.tokenize(q.lower())[0:5])
print("Sequence : ",seq)
print("Next possible words :",predict_completions(seq,5))

correct sentence :  Do your work by your own instead of depending on someone 
Sequence :  do your work by your
do
your
work
by
your
